# exporting the Breton example sentences

In [6]:
import datetime, tqdm, re, time
from mwclient import Site
# import dominate, pypandoc
# from dominate.tags import *
# from dominate.util import raw
import mwparserfromhell, mwcomposerfromhell

In [2]:
#!pip install --upgrade ipykernel
#!pip install mwcomposerfromhell
#%pip install pypandoc

In [ ]:
print('connecting...')
site = Site('arbres.iker.cnrs.fr/', path='/')
outf = open('problemPages.html','w')

# images = str(list(site.allimages()))
#print(images)
#print(dir(images[0]))
#print(images[0].name)
#qsdf
allpages = list(site.allpages())
print('got all', len(allpages), 'pages')

connecting...
got all 8131 pages


### getting the content...
this is slow!

In [8]:
t = time.time()
pages = {}
sources = {}
for p in tqdm.tqdm(allpages):
	if not('/documentation' in p.name or '/docname' in p.name or 'Module:' in p.name):
		pages[p.name] = p.text()
	if '(' in p.name:
		sources[p.name] = p.text()        
#print('extraction done. it took', time.time()-t, 'seconds.')

  0%|▏                                                                               | 17/8131 [00:06<54:20,  2.49it/s]


KeyboardInterrupt: 

In [9]:
import pickle
with open("Dict.txt", "rb") as myFile:
    pages = pickle.load(myFile)

In [10]:
with open("Sources.txt", "rb") as file:
    sources = pickle.load(file)

In [11]:
# print(', '.join(pages.keys()))
open('pageTitles.txt','w', encoding="utf-8").write('\n'.join(pages.keys()))
print('extracted',len(pages),'pages')

extracted 8068 pages


In [12]:
nbpretty=0
for name,text in pages.items():
	nbpretty += text.count('class="prettytable"')
print('found',nbpretty,'examples')

found 13422 examples


In [38]:
"""
{| class="prettytable"
|(1)|| N'eo || ket || dleet || koduiñ || hag || evañ || er || memes || amzer.
|-
||| [[ne]] [[COP|est]] || [[ket|pas]] || [[dleout|dû]] || conduire || [[&|et]] || [[evañ|boire]] || [[P.e|en]].[[art|le]] || [[memes|même]] || temps
|-
|||colspan="10" | 'On ne doit pas boire et conduire en même temps /Il ne faut pas boire et conduire' 
|-
|||||||||colspan="10" | ''Lesneven/Kerlouan'', [[Y. M. (04/2016)]]
|}
"""
references = {} # keeps track of the references of the sentences: ref:[sentence]
sentences = {} # keeps the actual texts
dic = {} # token to lemma, pos, gloss
errorout = open('breton.errors.txt','w')
nopos = open('no_pos.txt','w', encoding='utf-8')
currentPage=''

unknownPOS = []
repretty = re.compile(r'\{\|.*?class="prettytable"((\n|.)*?)\|\}', re.MULTILINE)
#CHANGEMENT 24/05: nouvelle liste de POS
rePOS = re.compile(r'verbes|auxiliaires|copules|adverbes|complémenteurs|conjonctions|prépositions|adjectifs|noms|particules verbales|interjections|postpositions|déterminants|quantifieurs|pronoms|noms propres', re.IGNORECASE)
reDialect = re.compile(r'léonard|cornouaillais|trégorrois|vannetais', re.IGNORECASE)
reType = re.compile(r'ouvrages de recherche|références de corpus|élicitations|ouvrages pédagogiques', re.IGNORECASE)
# print(repretty.search(pages['Konduiñ']))
reapost = re.compile(r"(.*?')(.*)")
relinks = re.compile(r"\[\[(.*?)\]\]")

verbose = False
# print(pages['Konduiñ'])

def getFirstPos(title):
	if not title: return '?'
	title = title[0].upper()+title[1:]
	# print('getting pos for',title, 'in pages:',title in pages)
	mpos=''
	if title.isupper():
		mpos = title        
	if title in pages:
		# print(pages[title][:222])
		if '#REDIRECTION' in pages[title]:
			newtitle = relinks.search(pages[title]).group(1)
			newtitle = newtitle[0].upper()+newtitle[1:]
			if newtitle in pages:
				wikicode = pages[newtitle]
			else: 
				# print(8888,newtitle, newtitle in pages)
				newtitle = newtitle[0].upper()+newtitle[1:].replace(' ','_')
				if newtitle in pages:
					wikicode = pages[newtitle]
				else:
					newtitle = newtitle.split(',')[0]
					newtitle = newtitle[0].upper()+newtitle[1:]
					if newtitle in pages:
						wikicode = pages[newtitle]
					else:
						if verbose: print('strange redirect to page that does not exist:',newtitle)
						errorout.write('\n'+currentPage+': strange redirect to page that does not exist: '+newtitle+'\n')
#CHANGEMENT
		else:
			wikicode = pages[title]
			if re.findall(r"\[\[Category:.*\|", wikicode):
				cats = re.findall(r"\[\[Category:.*\|", wikicode)
				list_cats = []
				for cat in cats:
					if rePOS.search(cat) and rePOS.search(cat).group(0) not in list_cats:
						list_cats.append(rePOS.search(cat).group(0).rstrip('s'))                                   
						if len(list_cats) == 1:                                          
							mpos = list_cats[0]
						elif len(list_cats) >0: 
							mpos = "-".join(list_cats)
		if mpos:
			pos = mpos
#FIN CHANGEMENT
		else:
			pos = "?"
			nopos.write(title+'\n')
	else:
		pos = "?"
		nopos.write(title + '\n')        
	# print('pos',pos)
	return pos

def trans2lemposgloss(tr, t):
	# print('tr:',tr)
	m = relinks.search(tr)
	if m:
		# print('got links', m.group(1))
		if ']].[[' in tr:
			# print('amalgame!')
			lems, glosss, poss = [], [], []
			for m in relinks.finditer(tr):
				if '|' in m.group(1):
					# print('got lemma')
					ll = m.group(1).split('|')[0]
					lems += [ll]
					glosss += [m.group(1).split('|')[1]]
					poss += [getFirstPos(ll)]
			lem = '.'.join(lems)
			gloss = '.'.join(glosss)
			pos = '.'.join(poss)
		elif '|' in m.group(1):
			# print('got lemma')
			lem = m.group(1).split('|')[0]
			gloss = m.group(1).split('|')[1]
			pos = getFirstPos(lem)
			if pos=='?':
				pos = getFirstPos(t)
		else:
			# case for example 'ne'
			lem=tr
			gloss=tr
			pos = getFirstPos(t)
			if pos=='?':
				pos = getFirstPos(tr)
	else:
		# print('just a word')
		lem = t
		gloss = tr
		pos = getFirstPos(t)
		if pos=='?':
			pos = getFirstPos(tr)
	# print('__lem, pos, gloss:', lem, pos, gloss)
	return lem, pos, gloss

def cleanToken(t):
	t = t.replace("'''", "")
	t = re.sub(r'</?font.*?>','',t)
	t = re.sub(r'<sup>','_',t)
	t = re.sub(r'</sup>','',t)
	return t.strip()

def has_numbers(inputString):
	return any(char.isdigit() for char in inputString)


def analyzePage(title):
	currentPage=title
	mpos = rePOS.search(pages[title])
	pagepos = None
	conlls=[]
	if mpos:
		pagepos = mpos.group(0).lower()
		# print('pagepos',pagepos)
	for m in repretty.finditer(pages[title]):
		# print('\n\n_______________',m.group(1))
		tokens, trans = [], []
		translation, source, dialect, location, texttype, l = '', '', '', '', '', ''
		for li in m.group(1).split('\n'): # for every line of the table
			if li[:2]=='|(':
				tokens = [cleanToken(t) for t in li.split('||')[1:]]
			if li[:3]=='|||':
				if not trans:
					trans = [cleanToken(tr) for tr in li[3:].split('||')]
					# print(len(tokens), len(tokens)==len(trans))
				elif 'colspan=' in li:
					if not translation:
						translation = li.split('|')[-1]
					elif not source:                      
						source = li.split('|')[-1].replace('[[','').replace(']]','').replace("''",'').replace(' ','')                       
						if re.findall("''.*''", li):
							#source_brut = re.findall("''.*'',", li)[0]                      
							if re.search(r"\)''", li):           
								location = re.findall(r"\(([^\)]+)\)", li)[0]
								if has_numbers(location):
									location = ''                         
								#dialect = re.findall(r"''(.*?)\s*\(", li)[0]
								#dialect.lstrip("''").rstrip("(")                                
							#else:
								#dialect = re.findall(r"''(.*?)''", li)[0]
						if re.search(r"\[\[(.*?)\]\]", li):                 
							l = re.search(r"\[\[(.*?)\]\]", li).group(0)        
							l = str(l).split('|')[0].replace('[[', '').replace(']]', '').replace("'", "")         
							if l in sources:
								bib_source = sources[l]
								if re.findall(r"\[\[Category:.*\|", bib_source):
									bibl = re.findall(r"\[\[Category:.*\|", bib_source)
									for ele in bibl:
										tdialect = reDialect.search(ele)
										if tdialect:
											dialect = tdialect.group(0).capitalize().replace("[[Category:", "").replace("|", "")
										ttype = reType.search(ele)
										if ttype:
											texttype = ttype.group(0).capitalize().replace("[[Category:", "").replace("|", "") 
                                    
						# print(111,source)
		# print(tokens, trans, translation, source)
		if not( tokens and trans and translation and source): # if one is missing
			if verbose: print('Houston, we got a problem: The number of tokens and the number of transcriptions differ:', title) 
			errorout.write('\n'+currentPage+': The number of tokens and the number of transcriptions differ: '+m.group(1)+'\n')
			continue
		
		# construction of the conll:
		text = ' '.join(tokens)
		references[source] = references.get(source,[])
		if text in references[source]:
			ind = references[source].index(text)+1
		else:
			references[source]+=[text]
			ind = 1
		sentences[text] = sentences.get(text,0)+1
		conllis = ['sent_id = '+source+'__'+str(ind)]
		conllis += ['text = '+text]
		conllis += ['text_fr = '+translation]
		if dialect :
			conllis += ['dialect = ' + dialect]
		if location :
			conllis += ['location = ' + location]
		if l :
			conllis += ['source = ' + l] 
		if texttype :
			conllis += ['texttype = ' + texttype]   
		extrai = 1
		for i,t in enumerate(tokens):
			if not t:
				errorout.write('\n___ '+currentPage+': strange token nr '+str(i)+' in '+m.group(1))
				t='???'
			if len(trans)<=i:
				errorout.write('\n___ '+currentPage+': strange token nr '+str(i)+' in '+m.group(1))
				tr='???'
			else:
				tr = trans[i]
			# print('_________t, tr', t, tr)
			
			if t.count("'")==1 and tr.count(']] [[')==1:
				# 2 tokens
				# case N'eo -- [[ne]] [[COP|est]] 
				t1 = reapost.search(t).group(1)
				tr1 = relinks.search(tr).group(1)
				lem, pos, gloss = trans2lemposgloss(tr1, t1)
				dic[t1] = dic.get(t1, [])+[(lem,pos,gloss)]
				eles = [str(i+extrai),t1, lem, pos]+5*['_']+['Gloss='+gloss]
				conllis += ['\t'.join(eles)]

				extrai += 1
				t2 = reapost.search(t).group(2).strip()
				tr2 = tr.strip().split(']] ')[1]
				lem, pos, gloss = trans2lemposgloss(tr2, t2)
				dic[t2] = dic.get(t2, [])+[(lem,pos,gloss)]
				eles = [str(i+extrai),t2, lem, pos]+5*['_']+['Gloss='+gloss]
				conllis += ['\t'.join(eles)]
				# print(222,eles)
			elif t[-1] in ".,;!?":
				# last character = punctuation
				lem, pos, gloss = trans2lemposgloss(tr, t[:-1])
				
				if pos=='?' and t[:-1].lower()==title.lower() and pagepos:
					# couldn't find a pos, maybe it's the word of the page itself?
					pos = pagepos
				dic[t[:-1]] = dic.get(t[:-1], [])+[(lem,pos,gloss)]
				eles = [str(i+extrai),t[:-1], lem, pos]+5*['_']+['Gloss='+gloss]
				conllis += ['\t'.join(eles)]
				extrai += 1
				dic[t[-1]] = dic.get(t[-1], [])+[(t[-1], 'PUNCT','punct')]
				eles = [str(i+extrai),t[-1], t[-1], 'PUNCT']+5*['_']+['Gloss=punct']
				conllis += ['\t'.join(eles)]
			else:
				# normal case
				lem, pos, gloss = trans2lemposgloss(tr, t)

				if pos=='?' and t.lower()==title.lower() and pagepos:
					# couldn't find a pos, maybe it's the word of the page itself?
					pos = pagepos
				dic[t] = dic.get(t, [])+[(lem,pos,gloss)]
				eles = [str(i+extrai),t, lem, pos]+5*['_']+['Gloss='+gloss]
				conllis += ['\t'.join(eles)]
		# print('\n'.join(conllis))
		conlls += ['\n'.join(conllis)]
	return conlls

print('____________\n')
for conll in analyzePage('Souezhañ'):
	print(conll,'\n')

____________

sent_id = Milin(1922:402)__1
text = Ar plac'h yaouank, ne ouie doare eus a netra, a zo souezet.
text_fr =  'La jeune femme, qui ne savait rien de rien, est étonnée.'
dialect = Léonard
location = Saint Pol de Léon
source = Milin (1922)
texttype = Références de corpus
1	Ar plac'	Ar plac'	?	_	_	_	_	_	Gloss=art|le
2	h	plac'h	nom	_	_	_	_	_	Gloss=fille
3	yaouank	yaouank	adjectif	_	_	_	_	_	Gloss=jeune
4	,	,	PUNCT	_	_	_	_	_	Gloss=punct
5	ne ouie	[[ne]]_[[1]] [[gouzout|savait]]	?	_	_	_	_	_	Gloss=[[ne]]_[[1]] [[gouzout|savait]]
6	doare	doare	nom	_	_	_	_	_	Gloss=façon
7	eus a	eus	?	_	_	_	_	_	Gloss=de
8	netra	netra	quantifieur	_	_	_	_	_	Gloss=rien
9	,	,	PUNCT	_	_	_	_	_	Gloss=punct
10	a zo	[[R]]_[[1]] [[zo|est]]	?	_	_	_	_	_	Gloss=[[R]]_[[1]] [[zo|est]]
11	souezet	souezet	?	_	_	_	_	_	Gloss=étonné
12	.	.	PUNCT	_	_	_	_	_	Gloss=punct 

sent_id = ArBorgn(2011:65)__1
text = Ma zud a veze souezhet dezhoñ bout yen sklas.
text_fr =  'Mes parents étaient surpris qu'il soit gelé.'
dialect = Vann

# testing

In [11]:
print(dic)

{"Ar plac'": [("Ar plac'", '?', 'art|le')], 'h': [("plac'h", 'nom', 'fille')], 'yaouank': [('yaouank', 'adjectif', 'jeune')], ',': [(',', 'PUNCT', 'punct'), (',', 'PUNCT', 'punct')], 'ne ouie': [('[[ne]]_[[1]] [[gouzout|savait]]', '?', '[[ne]]_[[1]] [[gouzout|savait]]')], 'doare': [('doare', 'nom', 'façon')], 'eus a': [('eus', 'préposition', 'de')], 'netra': [('netra', 'nom', 'rien')], 'a zo': [('[[R]]_[[1]] [[zo|est]]', '?', '[[R]]_[[1]] [[zo|est]]')], 'souezet': [('souezet', '?', 'étonné')], '.': [('.', 'PUNCT', 'punct'), ('.', 'PUNCT', 'punct'), ('.', 'PUNCT', 'punct')], 'Ma': [('POSS', 'nom', 'mon')], 'zud': [('tud', 'nom', 'parents')], 'a veze': [('[[R]] [[COP|était]]', '?', '[[R]] [[COP|était]]')], 'souezhet': [('souezhet', '?', 'étonné')], 'dezhoñ': [('da.pronom incorporé', 'préposition.nom', 'à.lui')], 'bout': [('bezañ', 'verbe', 'être')], 'yen': [('yen', 'adjectif', 'froid')], 'sklas': [('intensifieur', 'adjectif', 'glacé')], 'So-se': [('seurt', 'nom', 'sorte')], 'en dwe': [('

In [56]:
for conll in analyzePage('Memes'):
	print(conll,'\n')


ok
ok
ok
ok
ok
ok
ok
sent_id = Denez(1984:73)__1
text = Ha merc'hed Treboull noe ket tre ar memes taol dorn da lakaat o c'hoef.
text_fr =  'Et les filles de Tréboul n'avaient pas la même façon de mettre leur coiffe'.
dialect = Douarnenez
1	Ha merc'	Ha merc'	?	_	_	_	_	_	Gloss=&|et
2	hed	merc'h.-ed (PL.)	nom.nom	_	_	_	_	_	Gloss=femme.s
3	Treboull	nom propre	nom	_	_	_	_	_	Gloss=Tréboul
4	noe	kaout	verbe	_	_	_	_	_	Gloss=avait
5	ket	ket	adverbe	_	_	_	_	_	Gloss=pas
6	tre	tre	adjectif	_	_	_	_	_	Gloss=tt.à.fait
7	ar memes	art	article	_	_	_	_	_	Gloss=le
8	taol dorn	taol (M.)	nom	_	_	_	_	_	Gloss=coup
9	da	da	préposition	_	_	_	_	_	Gloss=pour
10	lakaat	lakaat	verbe	_	_	_	_	_	Gloss=mettre
11	o	POSS	nom	_	_	_	_	_	Gloss=leur
12	c'hoef	koef	nom	_	_	_	_	_	Gloss=coiffe
13	.	.	PUNCT	_	_	_	_	_	Gloss=punct 

sent_id = Bouzeg(1986:II)__1
text = Ne wriont ket war ar memes liñser.
text_fr =  'Ils ne sont pas du même bord politique.' 
dialect = Cornouaillais
location = Riec
1	Ne	[[ne]]_[[1]]	complémenteur	_	_	

In [57]:
print(dic)

{'Diwallit': [('diwall', 'verbe', 'méfiez')], 'mat': [('mat', 'adjectif', 'bien'), ('mat', 'adjectif', 'bon')], 'diouzh': [('diouzh', 'préposition', 'de'), ('art', 'article', 'le'), ('diouzh', 'préposition', 'P'), ('diouzh', 'préposition', 'à'), ('diouzh', 'préposition', 'de'), ('diouzh', 'préposition', 'de'), ('diouzh', 'préposition', 'de')], 'ar bilpouzed': [('art.pilpous.-ed (PL.)', 'article.?.nom', 'le.hypocrite.s')], '!': [('!', 'PUNCT', 'punct'), ('!', 'PUNCT', 'punct'), ('!', 'PUNCT', 'punct')], 'hag e': [('&', 'conjonction', 'et')], 'veze': [('vez', 'verbe', 'était')], 'troet': [('treiñ.-et (Adj.)', 'verbe.verbe', 'tourn.é')], "dac'h": [("dac'h", 'préposition', 'selon')], 'ar roud-avel': [('art', 'article', 'le')], 'a veze ': [('[[R]]_[[1]] [[vez|était]]', '?', '[[R]]_[[1]] [[vez|était]]')], 'An avel': [('art', 'article', 'le')], 'a teu': [('[[R]] [[dont|vient]]', '?', '[[R]] [[dont|vient]]')], 'direct': [('direct', 'nom', 'direct')], 'douh': [('douh', '?', 'de')], 'kement': [(

In [39]:
def writeDic():
	out = open('breton.dic.tsv','w', encoding="utf-8")
	for t, lpg in sorted(dic.items()):
		lpg = sorted(set(lpg))
		
		lines = ['\t'.join([t,l,p,g]) for l,p,g in lpg]
		out.write('\n'.join(lines)+'\n')
writeDic()

# trying to run through the whole site:

In [40]:

print(len(pages))
dic = {}
errorout = open('breton.errors.txt','w', encoding="utf-8")
nopos = open('no_pos.txt', 'w', encoding='utf-8')
for title in tqdm.tqdm(pages):
	# print('____________',title)
	conlls = analyzePage(title)
	if conlls:
		open('bretonconlls/'+re.sub(r'\W','_',title)+'.conllu','w', encoding="utf-8").write('\n\n'.join(conlls))
writeDic()

8068


100%|█████████████████████████████████████████████████████████████████████████████| 8068/8068 [00:13<00:00, 585.36it/s]


In [26]:
print(len(sentences), sum(sentences.values()))









































8216 25627
